---
title: "The dice rolls problem"
author: "Damien Martin"
date: "2024-05-22"
categories: [interview,puzzles]
---

# Problem

I came across [this](https://www.interviewcake.com/question/python/simulate-5-sided-die) Interview Cake problem:

> You have a function rand7() that generates a random integer from 1 to 7. Use it to write a function rand5() that generates a random integer from 1 to 5.
> 
> rand7() returns each integer with equal probability. rand5() must also return each integer with equal probability.

They had a solution, but also had a callout that I thought was interesting.

# Solution

This is a pretty standard problem, where the trick is that you will have to go to some form of rejection sampling. If you roll $N$ dice, you can generated $5^N$ different permutations of outcomes, which will never be divisible by 7 -- the only prime in the factorization of $5^N$ is 5, and 7 is also prime.

So a basic solution is

- Roll the die N times (or call `rand5()` 2 times)
- Construct a number from 0 to 24. The easiest way is to take `number = (roll1 - 1) * 5 + (roll2 -1)`. Essentially we are treating each roll as generating a digit, base 5, but shifting to get numbers from 0-4 from the rolls instead of 1-5.
- `25 % 7` is 4, so we don't go evenly. 
  - If `number` is from 0 to 20 inclusive, or 21 of the possible 25 answers, we return `(number % 7) + 1`. Here number occurs uniformly from 0 to 6.
  - We will take the last 4 numbers (21, 22, 23, 24) as "reject" and call the function again, so that we can maintain an even distribution.

If we didn't want to take more space on the stack, you could write the function like this

```python
def rand7():
    while True:
        number = (rand5() - 1) * 5 + (rand5() - 1)
        if number <= 20:
            return (number % 7) + 1
```

We can test this function using
```python
import numpy 

def rand5():
    # an implementation of rand5; randint is inclusive on lower and exclusive on upper
    return np.random.randint(1, 6)
```

The worst case running time is infinite, but a single pass through the loop rejects the answer 4/25, and requires running again. The acceptance or success rate is 21/25, and so the _expected_ number of loops is 25/21 ~ 1.2.

## Wrinkle

As far as I can tell, this pretty much matches the Interview Cake solution. 

The format of Interview Cake is to give hints, and one of the hints suggested that "Did you know you can do this with only two calls to `rand5()` (per loop), not 3?"

This seemed to be odd, and very prescriptive (i.e. guess the answer I am thinking of) instead of "let's think of the best answer". Let's make the assumption that calls to `rand5()` are really expensive. Is it really best to minimize the number of calls in the loop?

As we saw above, we are expected to run the loop above 1.2 times, and `rand5()` is called twice per loop. Ergo, the number of expected calls is 2.4. Since this is less than 3 (the minimum we could do with 3 calls per loop is 3 calls.)

Still, it is interesting to know what 3 calls in a loop would look like:

- We would be generating a number from 0 to $5**3 - 1$, or 125 possible options.
- We have `125 % 7= 6`, so the probability of rejection is 6 / 125; probability of acceptance is (125 - 6)/125
- The expected number of runs is 125/119 ~ 1.05

The expected number of calls to `rand5()` would be 1.05 x 3 = 3.15

In general, letting
- $n$ be the number of times `rand5()` is called
- $N=5^N$ as the number of options 
- $m = N \mod 7$ as the modulus
We have a rejection probability of `m/N`, an acceptance probability per loop of `(N-m)/N`, and an expected number of loop executions of `N/(N-m)`. The expected number of calls to `rand5()` is $n N / (N-m) \leq nN / (N - 6)$  as 6 is the worst case sceanario for the modulus.

We can get a rough feel for how the number of calls to `rand5()` grows as you make each loop more efficent by calling more times in a loop (exchanging a lower rejection rate per loop for more work per loop). The tradeoff is roughly `n(1 + 6/N)`, from expanding the $N/(N - 6) = 1/(1-6/N) = (1 + 6/N + \ldots)$.

So the interview cake solution of only using two `rand5()` calls was correct:

- It yields an infinite worst case (but so does every solution)
- It yields a lower minumum number of calls (2), and a lower expected number of calls (2.4) than rolling three
- It yields a higher variance, though (not shown)

## Going one step deeper

Let's say that we `rand5()` really was the long poll in the tent, and we could only call it `n` times. What would be the probability of a timeout?

| n dice |  P(loops > 1) | P(loops > 2) | P(loops > 3) |
|--------|---------------|--------------|--------------|
|   2    |         16.0% |        2.56% |       0.41%  |
|   3    |          4.8% |        0.23% |       0.011% | 
 

So if we had time to roll up to 6 die, we could do 3 loops of 2 dice (and have a failure rate of 0.41%), or up to two loops of a 3 roll (with a 0.23% failure rate). If we did $n=6$, there would only be a 0.0064% failure rate (cut a lot more guaranteed work per roll).